In [1]:
pip list

Package                 Version
----------------------- -----------
absl-py                 2.1.0
asttokens               2.4.1
astunparse              1.6.3
certifi                 2024.12.14
charset-normalizer      3.4.1
colorama                0.4.6
comm                    0.2.2
contourpy               1.3.0
cycler                  0.12.1
debugpy                 1.8.6
decorator               5.1.1
executing               2.1.0
flatbuffers             24.12.23
fonttools               4.54.1
gast                    0.6.0
google-pasta            0.2.0
grpcio                  1.68.1
h5py                    3.12.1
idna                    3.10
ipykernel               6.29.5
ipython                 8.27.0
jedi                    0.19.1
joblib                  1.4.2
jupyter_client          8.6.3
jupyter_core            5.7.2
keras                   3.7.0
kiwisolver              1.4.7
libclang                18.1.1
Markdown                3.7
markdown-it-py          3.0.0
MarkupSafe         

In [116]:
import tensorflow as tf
import os # used to navigate the file structure

In [117]:
import cv2
import imghdr
from matplotlib import pyplot as plt

In [ ]:
data_dir = 'data'

os.listdir(os.path.join(data_dir, 'happy'))

In [47]:
image_exts = ['jpeg', 'jpg', 'bmp', 'png']

In [114]:
for image_class in os.listdir(data_dir): 
    for image in os.listdir(os.path.join(data_dir, image_class)):
        print(image)

In [ ]:
# how to read an image using opencv
img = cv2.imread(os.path.join('data', 'happy', '960x0.jpg'))
print(img.shape)
plt.imshow(img)

In [50]:
for image_class in os.listdir(data_dir): 
    for image in os.listdir(os.path.join(data_dir, image_class)):
        image_path = os.path.join(data_dir, image_class, image)
        try: 
            img = cv2.imread(image_path)
            tip = imghdr.what(image_path)
            if tip not in image_exts: 
                print('Image not in ext list {}'.format(image_path))
                os.remove(image_path)
        except Exception as e: 
            print('Issue with image {}'.format(image_path))

In [ ]:
# loading data
tf.data.Dataset??


In [52]:
import numpy as np

In [ ]:
tf.keras.utils.image_dataset_from_directory??

In [54]:
# this expression build a dataset from your 'data'
# building the data pipeline
data = tf.keras.utils.image_dataset_from_directory('data')

Found 180 files belonging to 2 classes.


In [55]:
# accessing our data pipeline
data_iterator = data.as_numpy_iterator()

In [56]:
batch = data_iterator.next()

In [61]:
batch[0].min()


np.float32(0.0)

In [ ]:
fig, ax = plt.subplots(ncols=4, figsize=(20,20))
for idx, img in enumerate(batch[0][:4]):
    ax[idx].imshow(img.astype(int))
    ax[idx].title.set_text(batch[1][idx])

In [67]:
batch[0].max()


np.float32(255.0)

In [68]:
data = data.map(lambda x,y: (x/255, y))

In [ ]:
data.as_numpy_iterator().next()[0]

In [73]:
data.as_numpy_iterator().next()[0].min()

np.float32(0.0)

In [91]:
len(data)

6

In [92]:
train_size = int(len(data)*.7)
val_size = int(len(data)*.2)
test_size = int(len(data)*.1)+1

In [96]:
train = data.take(train_size)
val = data.skip(train_size).take(val_size)
test = data.skip(train_size+val_size).take(test_size)

In [97]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout


In [99]:
model = Sequential()

In [100]:
model.add(Conv2D(16, (3,3), 1, activation='relu', input_shape=(256,256,3)))
model.add(MaxPooling2D())

model.add(Conv2D(32, (3,3), 1, activation='relu'))
model.add(MaxPooling2D())

model.add(Conv2D(16, (3,3), 1, activation='relu'))
model.add(MaxPooling2D())

model.add(Flatten())

model.add(Dense(256, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

c:\Users\Administrator\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [101]:
model.compile('adam', loss=tf.losses.BinaryCrossentropy(), metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
hist = model.fit(train, epochs=20, validation_data=val)

In [ ]:
fig = plt.figure()
plt.plot(hist.history['loss'], color='teal', label='loss')
plt.plot(hist.history['val_loss'], color='orange', label='val_loss')
fig.suptitle('Loss', fontsize=20)
plt.legend(loc="upper left")
plt.show()

In [ ]:

fig = plt.figure()
plt.plot(hist.history['accuracy'], color='teal', label='accuracy')
plt.plot(hist.history['val_accuracy'], color='orange', label='val_accuracy')
fig.suptitle('Accuracy', fontsize=20)
plt.legend(loc="upper left")
plt.show()

In [107]:
from tensorflow.keras.metrics import Precision, Recall, BinaryAccuracy

In [108]:
pre = Precision()
re = Recall()
acc = BinaryAccuracy()

In [109]:
for batch in test.as_numpy_iterator(): 
    X, y = batch
    yhat = model.predict(X)
    pre.update_state(y, yhat)
    re.update_state(y, yhat)
    acc.update_state(y, yhat)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 670ms/step


In [112]:
print(f'Precision:{pre.result().numpy()}, Recall:{re.result().numpy()}, Accuracy:{acc.result().numpy()}')

Precision:1.0, Recall:1.0, Accuracy:1.0


In [ ]:
# saving the model